## Bell Tests for Quantum and Classical Variables

In [ ]:
import random, math

### Bell test for classical variables

In this notebook you will investigate how quantum variables (based on qubits) differ from standard ones (based on bits).

We'll do this by creating a pair of variables, which we will call `A` and `B`. These could be represented by any kind of data structure, and could be initialized by any kind of process. This could be deterministic or random. It could initialize the two variables in a correlated way, or they could be completely independent. Whatever you choose do do, do it in the function below.

As an example, I define `A` and `B` to be partially correlated random floating point numbers.

In [ ]:
def setup_variables ():
    
    ### Replace this section with anything you want ###
    
    r = random.random()
    
    A = r*(2/3)
    B = r*(1/3)
    
    ### End of section ###
    
    return A, B

We then define a hashing function that can be applied to either variable to produce a bit. So when given either one of the variables as an input, this function must return a bit value as output.

The hashing function must also be capable of performing two different types of hash. The hash type must therefore also be supplied as an input to the function.

To be consistent with the rest of the program, the two possible hash types should be called `'H'` and `'V'`. Also, the output must be in the form of a single value bit string: either `'0'` or `'1'`.

As an example, I created bits by comparing `A` and `B` to a certain value. The output is `'1'` if they are under that value, and `'0'` otherwise. The value used depends on the type of hash.

In [ ]:
def hash2bit ( variable, hash ):
    
    ### Replace this section with anything you want ###
    
    if hash=='V':
        bit = (variable<2/3)
    elif hash=='H':
        bit = (variable<1/3)
        
    bit = str(int(bit))
    
    ### End of section ###
        
    return bit

Once these are defined, there are four quantities we wish to calculate: `P['HH']`, `P['HV']`, `P['VH']` and `P['VV']`.

Let is focus on `P['HV']` as an example. This is the probability that the bit value derived from an `'H'` type hash on `A` is the same as that from a `'V'` type has on `B`. We will estimate this probability by sampling many times and determining the fraction of samples for which the corresponding bit values agree.

The other probabilities are defined similarly: `P['HH']` comapres a `'H'` type hash on both `A` and `B`, `P['VV']` compares a `V` type hash on both, and `P['VH']` compares a `V` type hash on `A` with a `H` type has on `B`.

These probabilities are calculated in the following function, which returns all the values of `P` in a dictionary. The parameter `shots` is the number of samples we'll use.

In [ ]:
shots = 1024
def calculate_P ( ):
    
    P = {}
    for hashes in ['VV','VH','HV','HH']:
        
        # calculate each P[hashes] by sampling over `shots` samples
        P[hashes] = 0
        for shot in range(shots):

            A, B = setup_variables()

            a = hash2bit ( A, hashes[0] ) # hash type for variable `A` is the first chacter of `hashes`
            b = hash2bit ( B, hashes[1] ) # hash type for variable `B` is the second chacter of `hashes`

            P[hashes] += (a!=b) / shots
 
    return P

Now let's actually calculate these values for the method we have chosen to set up and hash the variables.

In [ ]:
P = calculate_P()
print(P)

These values will vary slightly from one run to the next due to the fact that we only use a finite number of shots. To change them significantly, we need to change the way the variables are initiated, and/or the way the hash functions are defined.

No matter how these functions are defined, there are certain restrictions that the values of `P` will always obey.

For example, consider the case that `P['HV']`, `P['VH']` and `P['VV']` are all `0.0`. The only way that this can be possible is for `P['HH']` to also be `0.0`.

To see why, we start by noting that `P['HV']=0.0` means that `hash2bit ( A, H )` and `hash2bit ( B, V )` are never different So this means

    hash2bit ( A, H ) = hash2bit ( B, V )        (1)
    
From `P['VV']=0.0` and `P['VH']=0.0` we can similarly get

    hash2bit ( A, V ) = hash2bit ( B, V )        (2)
    
    hash2bit ( A, V ) = hash2bit ( B, H )        (3)
    
Putting (1) and (2) together implies that

    hash2bit ( A, H ) = hash2bit ( A, V )        (4)
    
Combining this with (3) gives

    hash2bit ( A, H ) = hash2bit ( B, H )        (5)

And if these values are always equal, the probability of them being different is zero. This is exactly what we set out to prove: `P['HH']=0.0`.

More generally, we can use the values of `P['HV']`, `P['VH']` and `P['VV']` to set an upper limit on what `P['HH']` can be. By adapting the [CHSH inequality](https://en.wikipedia.org/wiki/CHSH_inequality) (which is commonly used in Bell tests) we find that

$\,\,\,\,\,\,\,$ `P['HH']` $\, \leq \,$ `P['HV'] + P['VH'] + P['VV']`

This is not just a special property of `P['HH']`. It's also true for all the others: each of these probabilities cannot be greater than the sum of the others.

To test whether this logic holds, we'll see how well the probabilities obey these inequalities. Note that we might get slight violations due to the fact that our the `P` values aren't exact, but are estimations made using a limited number of samples.

In [ ]:
def bell_test (P):
    
    sum_P = sum(P.values())
    for hashes in ['VV','VH','HV','HH']:
        
        bound = sum_P - P[hashes]
        
        print("The upper bound for P["+hashes+"] is "+str(bound))
        print("The value of P["+hashes+"] is "+str(P[hashes]))
        if P[hashes]<=bound:
            print("The upper bound is obeyed :)\n")
        else:
            if P[hashes]-bound < 0.1:
                print("This seems to have gone over the upper bound, but only by a little bit :S\nProbably just rounding errors or statistical noise.\n")
            else:
                print("!!!!! This has gone well over the upper bound :O !!!!!\n")

In [ ]:
bell_test(P)

### Bell test for quantum variables

Now we are going to do the same thing all over again, except our variables `A` and `B` will be quantum variables. Specifically, they'll be the simplest kind of quantum variable: qubits.

Since you probably don't have a quantum computer handy, we'll have to borrow one of IBM's. So let's start by importing everything needed for creating a quantum program with QISKit, and setting up the API.

In [ ]:
from qiskit import ClassicalRegister, QuantumRegister
from qiskit import QuantumCircuit, execute
from qiskit import register, available_backends, get_backend

#import Qconfig and set APIToken and API url
import sys
sys.path.append("../") # go to parent dir
import Qconfig
qx_config = {
    "APItoken": Qconfig.APItoken,
    "url": Qconfig.config['url']}
register(qx_config['APItoken'], qx_config['url'])

When writing programs the programs for classical variables above, we didn't need to worry about manually doing memory allocation. But for quantum programs we work closer to the machine code level, and so we need to know exactly what all our qubits and bits are doing.

This is done in the function below. It sets up a program to be run on a five qubit device by declaring registers of five qubits and five bits. Thwo of the qubits are then chosen to be our variables  `A` and `B`. Two of the bits are also chosen to be where we write the corresponding has values `a` and `b`.

In [ ]:
def initialize_program ():
    
    
    qubit = QuantumRegister(2)
    bit = ClassicalRegister(2)
    program = QuantumCircuit(qubit, bit)
    
    A = qubit[0]
    B = qubit[1]
    a = bit[0]
    b = bit[1]

    return A, B, a, b, program

Now its time to set up the variables `A` and `B`. To do this you can go to the tutorial and pick the *Bell test* mode. Take the lines of QISKit code that you create and paste them into the empty function below.

In [ ]:
def setup_variables ( program, A, B ):
    
    pass # delete this line, and put your program here instead (and remeber to indent your code too)


For our two types of hash, we will simply use the outputs described in the tutorial. The hash of type `V` means extracting a bit from the qubit in the way described by the vertical blue box, and `H` means using the horizontal red box.

The `V` type hash is done with QISKit by using the command

    program.measure( qubit, bit )
    
The `H` type has cannot be done directly using QISKit. But we can do it indirectly using the `h` operation. This moves the contents of the red box to the blue box, and so doing it following by a `measure` operation is effectively an `H` type hash.

    program.h( qubit )
        
    program.measure( qubit, bit )
    
Note that this function has more inputs that its classical counterpart. We have to tell it the `bit` on which to write the result, and the quantum `program` that we will run to implement the process.

In [ ]:
def hash2bit  ( variable, hash, bit, program, ):
    
    if hash=='H':
        program.h( variable )
        
    program.measure( variable, bit )
        

Because our hashes correspond to the boxes in the tutorial, the same is true for the probabilities in `P`. Here's a table that shows which boxes correpond which values of `P`.

           |       |       |
           | (A,V) | (A,H) |
           |       |       |
    ───────|───────|───────|
           |       |       |
     (B,V) |P['VV']|P['HV']|
           |       |       |
    ───────|───────|───────|
           |       |       |
     (B,H) |P['VH']|P['HH']|
           |       |       |
    ───────|───────|───────|
    
For example, `P['HH']` is represented by the red box at the bottom left of the grid. This box is empty when `P['HH']=0.0`, full when `P['HH']=1.0`, etc.

The values of `P` are calculated in the function below. This is done by sending jobs to IBM and getting results which tell us how many of the samples gave each possible result. The results are given as a bit string, `string`, which IBM numbers from right to left. This means that the value of `a`, which corresponds to `bit[0]` is the first from the right

    a = string[-1]
    
and the value of `b` is right next to it at the second from the right

    b = string[-2]

The number of samples for this bit string is provided by the dictionary of results, `stats`, as `stats[string]`.

In [ ]:
def calculate_P ( backend ):
    
    P = {}
    program = {}
    for hashes in ['VV','VH','HV','HH']:

        A, B, a, b, program[hashes] = initialize_program ()

        setup_variables( program[hashes], A, B )

        hash2bit ( A, hashes[0], a, program[hashes])
        hash2bit ( B, hashes[1], b, program[hashes])
            
    # submit jobs
    job = execute( list(program.values()), backend, shots=shots )

    # get the results
    for hashes in ['VV','VH','HV','HH']:
        stats = job.result().get_counts(program[hashes])
        
        P[hashes] = 0
        for string in stats.keys():
            a = string[-1]
            b = string[-2]
            
            if a!=b:
                P[hashes] += stats[string] / shots

    return P

Now its time to choose and set up the actually device we are going to use. By default, we'll use a simulator. But if you want to use the real device, just replace the first line below with `device='ibmqx4'`.

In [ ]:
device = 'ibmqx_qasm_simulator'
backend = get_backend(device)
print(backend.status)

In [ ]:
P = calculate_P( backend )
print(P)

In [ ]:
bell_test( P )

If you prepared the state suggestion by the tutorial, you will have found a significant violation of the upper bound for `P['HH']`. So what is going on here? The chain of logic we based the Bell test on obviously doesn't apply to quantum variables. But why?

The answer is that there is a hidden assumption in that chain of logic. To see why, let's revisit point (4).

    hash2bit ( A, H ) = hash2bit ( A, V )        (4)
    
Here we compare the value we'd get from an `H` type of hash of the variable `A` with that for a `V` type hash.

For classical variables, this is perfectly sensible. There is nothing stopping us from calculating both hashes and comparing the results. Even if calculating the hash of a variable changes the variable, that's not a problem. All we need to do is copy it beforehand and we can do both hashes without any issue.

The same is not true for quantum variables. The result of the hashes is not known until we actually do them. It's only then that the qubit actually decides what bit value to give. And once it decides the value for one type of hash, we can never determine what it would have decided if we had used another type of hash. We can't get around this by copying the quantum variables either, because quantum variables [cannot be copied](https://en.wikipedia.org/wiki/No-cloning_theorem). This means there is no context in which the values `hash2bit(A,H)` and `hash2bit A,V)` are well-defined at the same time, and so it is impossible to compare them.

Another hidden assumption is that `hash2bit(A,hash)` depends only on the type of hash chosen for variable `A`, and not the one chosen for variable `B`. This is also perfectly sensible, since this exactly the way we set up the `hash2bit()` function. However, the very fact that the upper bound was violated does seem to imply that each variable knows what hash is  being done to the other, so they they can conspire to give very different behaviour when both have a `H` type hash.

Even so, the effect is subtle. It is impossible to determine which variable is affecting which: You can change the order in which the hashes are done, or [effectively do them at the same time](https://en.wikipedia.org/wiki/Loopholes_in_Bell_test_experiments#Communication,_or_locality), and you'll get the same results. This means it not clear whether we can say they affect each other( see [here](https://quantumcomputing.stackexchange.com/questions/2028/is-it-true-to-say-that-one-qubit-in-an-entangled-state-can-instantaneously-affec) for some discussion). But what we can say is that they are [contextual](https://en.wikipedia.org/wiki/Quantum_contextuality): to fully understand results from one variable, it is sometimes required to look at what was done to another.

All this goes to show that quantum variables don't always follow the logic we are used to. They follow different rules, the rules of quantum mechanics, which will allow us to find ways of performing computation in very different ways than we are used to.

To get more examples of this kind of effect, see [this notebook](https://github.com/QISKit/qiskit-tutorial/blob/master/reference/qis/entanglement_revisited.ipynb) in the QISKit tutorial.